# GLIF Example

This is the introductory GLIF Example shown at the beginning of the paper.

The fragment of this example is able to interpret the sentence "John runs and Mary doesn't run" as `(run john)∧¬(run mary)`

## Parsing

In [1]:
abstract TestGrammar = {
    cat
        S;     -- Sentence
        V;     -- Verb
        Npr;   -- Proper noun
        Conj;  -- Conjunction
    fun
        -- Positive sentence: "John runs"
        spos : Npr -> V -> S;
        -- Negative sentence: "John doesn't run"
        sneg : Npr -> V -> S;
        -- Connected sentence:
        -- "John runs and Mary doesn't run"
        sconj : S -> Conj -> S -> S;
        
        -- Verbs
        run, jump : V;
        -- Proper nouns
        john, mary : Npr;
        -- Conjunctions
        and, or : Conj;
}

Defined TestGrammar

In [2]:
concrete TestGrammarEng of TestGrammar = {
    oper
        -- The type of a verb
        VType : Type = { pos : Str ; neg : Str };
    lincat
        S = Str;     -- Sentence
        V = VType;   -- Verb
        Npr = Str;   -- Proper noun
        Conj = Str;  -- Conjunction
    oper
        -- Helper function to construct verb
        -- mkV "run" = { pos = "runs" ; neg = "doesn't run"}
        mkV : Str -> VType =
            \v -> { pos = v + "s" ; neg = "doesn't" ++ v };
    lin
        -- Positive sentence: "John runs"
        -- spos : Npr -> V -> S
        spos npr v = npr ++ v.pos;
        -- Negative sentence: "John doesn't run"
        -- sneg : Npr -> V -> S
        sneg npr v = npr ++ v.neg;
        -- Connected sentence:
        -- "John runs and Mary doesn't run"
        -- sconj : S -> Conj -> S -> S
        sconj s1 c s2 = s1 ++ c ++ s2;
        
        -- Verbs
        run = mkV "run";
        jump = mkV "jump";
        -- Proper nouns
        john = "John"; mary = "Mary";
        -- Conjunctions
        and = "and"; or = "or";
}

### Parsing test cases

In [3]:
parse "John runs"

spos john run

In [4]:
parse "John doesn't run"

sneg john run

In [5]:
parse "John runs and Mary doesn't run"

sconj (spos john run) and (sneg mary run)

In [6]:
parse "John doesn't jump or Mary runs"

sconj (sneg john jump) or (spos mary run)

## Semantics construction

In [7]:
theory plnq : ur:?LF =
    proposition : type ❘ # o ❙
    individual : type ❘ # ι ❙
    
    not : o ⟶ o         ❘ # ¬ 1 prec 100 ❙
    and : o ⟶ o ⟶ o     ❘ # 1 ∧ 2 prec 90 ❙
    
    or : o ⟶ o ⟶ o      ❘ # 1 ∨ 2 prec 80
                        ❘ = [a,b] ¬ (¬ a ∧ ¬ b) ❙
❚

theory plnqTest : ?plnq =
    // individuals ❙
    john : ι ❙ mary : ι ❙
    // verbs ❙
    run : ι ⟶ o ❙ jump : ι ⟶ o ❙
❚

Created theory plnq

In [8]:
view semantics : http://mathhub.info/comma/jupyter/TestGrammar.gf?TestGrammar -> ?plnqTest =
    S = o ❙             // Sentence ❙
    V = ι ⟶ o ❙         // Verb ❙
    Npr = ι ❙           // Proper noun ❙
    Conj = o ⟶ o ⟶ o ❙  // Conjunction ❙
    
    // Positive sentence: "John runs"
    // spos : Npr -> V -> S ❙
    spos = [npr, v] v npr ❙
    // Negative sentence: "John doesn't run"
    // sneg : Npr -> V -> S ❙
    sneg = [npr, v] ¬(v npr) ❙
    // Connected sentence:
    // "John runs and Mary doesn't run"
    // sconj : S -> Conj -> S -> S ❙
    sconj = [s1, conj, s2] conj s1 s2 ❙
    
    // Verbs ❙
    run = run ❙ jump = jump ❙
    // Proper nouns ❙
    john = john ❙ mary = mary ❙
    // Conjunctions ❙
    and = and ❙ or = or ❙
❚

Created view semantics

### Semantics construction test cases

In [9]:
parse "John runs" | construct

run john

In [10]:
parse "John doesn't run" | construct

¬(run john)

In [11]:
parse "John runs and Mary doesn't run" | construct

(run john)∧¬(run mary)

In [12]:
parse "John doesn't jump or Mary runs" | construct

¬(jump john)∨(run mary)